In [5]:
import os
import sys
import time
from datetime import date
from datetime import datetime
import time
import json
import platform

import stoneburner
#//*** Custom Functions:
#//*** mr_clean_text(input_series)
#//*** tokenize_series(input_series)
#//*** remove_stop_words(input_series)

# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#subreddits = ["wallstreetbets", "stocks", "wallstreetbetsOGs", "spacs", "investing", "pennystocks", "stockmarket", "options", "robinhoodpennystocks", "wallstreetbetsnew", "smallstreetbets"]
filepath = "./data/"
#filename_suffix = "_comments.csv.zip"
#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [6]:
#//*** Path to file aggregate
input_filename = "./data/processed_reddit_v4_1min.csv.zip"

input_filename = "./data/processed_reddit_v4_1min_test.pkl"
input_filename = "./ignore_folder/processed_reddit_v4_amc_1min_csv.zip"

raw_df = pd.read_csv(input_filename)

del raw_df['created_utc']
raw_df

,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
1,No he’s just being sarcastic,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
2,I was in contact with Schwab and they weren't ...,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
3,Reelection is never a concern for the communis...,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
4,"For American brokerages, up to $500,000 are SI...",2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
...,...,...,...,...,...,...,...
4355077,That's how you value a house? Cars are depreci...,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134
4355078,$dare yolo,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134
4355079,Back to normal not there yet,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134
4355080,💎👐🚀🚀🚀🚀,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134


In [7]:
raw_df['body'] = raw_df['body'].astype('str')

In [8]:
#//************************************************
#//*** Combine Body Text for each Stock Time
#//************************************************
tdf = pd.DataFrame()

print(len(raw_df['time'].unique()))
print("Starting Body Aggregation....")
start_time = time.time()
for group in raw_df.groupby('time'):
    
    #//*** Build aa temporary Series by grabbing the first line of the group[1] dataframe
    #//*** Only Body will update, the rest are static
    loop_series = group[1].iloc[0].copy()

    
    #//*** Combine all body strings into a single string, add to loop_series['body']
    loop_series['body'] = " ".join(list(group[1]['body']))
    #//*** Build a dataframe to hold the Series.
    #//*** Add Dataframe to tdf
    #print(loop_series)
    tdf = pd.concat ([tdf,pd.DataFrame(data=[loop_series], columns = list(group[1].columns))])

print(f"Built: {round(time.time()-start_time,2)}")
tdf

190326
Starting Body Aggregation....
Built: 2060.99


,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.09,4.090,4.0900,4.090,406
11,It’s a combination of options but I made big o...,2019-07-24 09:35:00,4.14,4.150,4.1300,4.150,10116
12,Idk about you but I know several people with H...,2019-07-24 09:36:00,4.14,4.170,4.1385,4.170,16449
13,Thanks.. Will check them out,2019-07-24 09:37:00,4.17,4.180,4.1540,4.175,15605
14,I'm waiting until Monday to sell a covered cal...,2019-07-24 09:40:00,4.19,4.195,4.1700,4.180,42314
...,...,...,...,...,...,...,...
4355042,It’s always possible but I like to give compan...,2021-06-30 19:19:00,214.78,214.780,214.7800,214.780,100
4355055,Did business in China so I am jaded When money...,2021-06-30 19:40:00,214.80,214.800,214.8000,214.800,176
4355064,"I automatically downvoted on the image alone, ...",2021-06-30 19:54:00,214.34,214.340,214.3200,214.320,811
4355065,"So how do I get my 12,000 I lost from these fo...",2021-06-30 19:55:00,214.15,214.150,214.0000,214.000,1499


In [10]:
#//*** Keep a copy of our work so far
output_filename = "./ignore_folder/1min_amc_combined.csv.zip"
tdf.to_csv(output_filename,compression='zip',index=False)

In [14]:
#//*** Merge with stock data
stock_df = pd.read_csv("./stocks/amc_1min.csv.zip")
stock_df['time'] = pd.to_datetime(stock_df['time'])

stock_df
tdf['time'] = pd.to_datetime(tdf['time'])
start_time = time.time()

print("Merging Stocks with tdf")
tdf = pd.concat([tdf,stock_df])
print(f"Merged {round(time.time()-start_time,2)}s ")

tdf.sort_values('time',inplace=True)
tdf

tdf.drop_duplicates(subset='time',inplace=True)

tdf['body'] = tdf['body'].replace(np.nan,"  ")
tdf

Merging Stocks with tdf
Merged 0.03s 


,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.090000,4.090000,4.090000,4.090000,406
292744,,2019-07-24 09:31:00,9.846181,9.912903,9.846181,9.912903,23448
292743,,2019-07-24 09:32:00,9.922435,9.951029,9.922435,9.922435,3394
292742,,2019-07-24 09:33:00,9.939591,9.998688,9.935302,9.998688,24294
292741,,2019-07-24 09:34:00,10.003453,10.017751,9.874776,9.960561,67195
...,...,...,...,...,...,...,...
4,,2021-07-12 19:56:00,41.250000,41.290000,41.220000,41.290000,10274
3,,2021-07-12 19:57:00,41.270000,41.350000,41.250000,41.330000,6832
2,,2021-07-12 19:58:00,41.330000,41.330000,41.250000,41.250000,14056
1,,2021-07-12 19:59:00,41.250000,41.280000,41.250000,41.250000,8974


In [15]:
tdf.sort_values('time',inplace=True, ignore_index=True)
tdf

,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.090000,4.090000,4.090000,4.090000,406
1,,2019-07-24 09:31:00,9.846181,9.912903,9.846181,9.912903,23448
2,,2019-07-24 09:32:00,9.922435,9.951029,9.922435,9.922435,3394
3,,2019-07-24 09:33:00,9.939591,9.998688,9.935302,9.998688,24294
4,,2019-07-24 09:34:00,10.003453,10.017751,9.874776,9.960561,67195
...,...,...,...,...,...,...,...
305093,,2021-07-12 19:56:00,41.250000,41.290000,41.220000,41.290000,10274
305094,,2021-07-12 19:57:00,41.270000,41.350000,41.250000,41.330000,6832
305095,,2021-07-12 19:58:00,41.330000,41.330000,41.250000,41.250000,14056
305096,,2021-07-12 19:59:00,41.250000,41.280000,41.250000,41.250000,8974


In [16]:
#//************************************************
#//*** Clean text body
#//************************************************

tdf['token'] = stoneburner.remove_stop_words(stoneburner.tokenize_series(stoneburner.mr_clean_text(tdf['body'],{"remove_empty":False})))
print("Done")

#//*** Detokenize the clean column as tfidf
tdf['tfidf'] = tdf['token'].apply(lambda x: ' '.join(x))


remove_empty False
Text Cleaning Time: 28.363612174987793
Tokenize Time: 72.05233216285706
Stop Words Time: 30.986631870269775
Done


In [17]:
#//*** Build tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

#loop_list.append(tfidf.fit_transform(input_df['tfidf']))
tfidf_matrix = []
tfidf_list = []
tfidf = TfidfVectorizer()

print("Starting tfidf....")
start_time = time.time()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(tdf['tfidf'])


print(f"Built: {round(time.time()-start_time,2)}")

print(tfidf_matrix)

Starting tfidf....
Built: 12.63
  (0, 109566)	0.081160688319722
  (0, 146149)	0.06839339903832685
  (0, 42248)	0.054400403905241264
  (0, 165799)	0.04803327613002852
  (0, 23812)	0.05563007411492619
  (0, 197647)	0.05984892594609786
  (0, 179280)	0.04325022654628121
  (0, 198029)	0.1253375562535772
  (0, 158581)	0.05010050757611861
  (0, 139266)	0.03739378248737743
  (0, 182759)	0.0409842803691115
  (0, 131146)	0.04688464375527015
  (0, 148053)	0.055113370620580554
  (0, 217808)	0.037209974840837746
  (0, 126902)	0.0318077304731081
  (0, 88907)	0.050611608165241836
  (0, 79865)	0.09426689581162874
  (0, 185438)	0.05654620857177913
  (0, 77463)	0.06544566385601087
  (0, 45388)	0.08921691789217077
  (0, 44485)	0.03418761325339383
  (0, 82234)	0.03724037585984199
  (0, 195021)	0.03615986773731347
  (0, 120516)	0.054551376969048396
  (0, 129498)	0.044382344016849824
  :	:
  (298912, 128888)	0.11951778170490236
  (298912, 3495)	0.15027379658460285
  (298912, 27682)	0.1266373081711504
  (298

In [20]:
import pickle
output_filename = "./data/processed_reddit_v4_1min_tfidf.pkl"

with open(output_filename, 'wb') as f:
    pickle.dump(tfidf, f)

output_filename = "./ignore_folder/processed_reddit_v4_1min_tfidf_matrix.pkl"
with open(output_filename, 'wb') as f:
    pickle.dump(tfidf_matrix, f)


In [21]:
output_filename = "./data/processed_reddit_v4_1min_amc_stock_comments.csv.zip"

#//*** Cleanup columns for storage
if 'body' in tdf.columns:
    del tdf['body']
    del tdf['tfidf']

tdf.to_csv(output_filename, compression='zip')

In [17]:
output_filename = "./data/processed_reddit_v4_1min_testdata_tfidf.pkl"
infile = open(output_filename,'rb')
t_tdif = pickle.load(infile)
infile.close()
print(t_tdif.get_params())
print(t_tdif.get_feature_names())


{'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': None, 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}


In [10]:
infile = open(output_filename,'rb')
t_tdif_matrix = pickle.load(infile)
infile.close()
print(t_tdif_matrix)

  (0, 8133)	0.08745808745100456
  (0, 11133)	0.07327893552266958
  (0, 2875)	0.05745903186031536
  (0, 12858)	0.05009304185394831
  (0, 1271)	0.059828786385647564
  (0, 15883)	0.06475547596357362
  (0, 14146)	0.04850398207814858
  (0, 15909)	0.0967335584413286
  (0, 12160)	0.055511687856221854
  (0, 10623)	0.037181240474684976
  (0, 14506)	0.04293238130669861
  (0, 10000)	0.051679345634732955
  (0, 11281)	0.05792547524640468
  (0, 17464)	0.03648588477406036
  (0, 9638)	0.03109594680876921
  (0, 6328)	0.05332397563086158
  (0, 5387)	0.08913682801099229
  (0, 14747)	0.05538498395581354
  (0, 5053)	0.06446319823408891
  (0, 3136)	0.08154009758065602
  (0, 3067)	0.03470696153058779
  (0, 5639)	0.036828048837588646
  (0, 15687)	0.03635195522166141
  (0, 9140)	0.0552597281316865
  (0, 9863)	0.04670937342947679
  :	:
  (5008, 8008)	0.21328624075614527
  (5008, 12039)	0.17110212787774884
  (5008, 6375)	0.14488634710955123
  (5008, 15717)	0.12363931585752246
  (5008, 7543)	0.18153131157240643
 